# BMI Prediction Using BRFSS Data

This notebook implements the complete pipeline for BMI prediction using the Behavioral Risk Factor Surveillance System (BRFSS) 2024 dataset, which is not included in this repo. The link to download this dataset is below:

https://www.cdc.gov/brfss/annual_data/annual_2024.html

James Simon (Project Conceptualization, Literature Review, Paper, Fairness, Feature Selection)
Shashank Joshi (Project Conceptualization, Model Training and Evaluation, Feature Selection, Paper)
Emma Yue (Graphs, Presentation, Data Pre-Processing, Risk Estimates)
Phani Yalamanchili (Presentation, Flowcharts, Feature Selection, Data Pre-Processing)


---

## Section 1: Imports & Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
from pathlib import Path
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import mutual_info_regression
from sklearn.inspection import permutation_importance
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    VotingRegressor,
    StackingRegressor
)
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    brier_score_loss,
    confusion_matrix,
    classification_report
)

warnings.filterwarnings('ignore')
np.random.seed(42)
plt.style.use('seaborn-v0_8')

In [ ]:
# WHO/CDC standards
BMI_CATEGORIES = [
    'Underweight',
    'Normal',
    'Overweight',
    'Obese Class I',
    'Obese Class II',
    'Obese Class III'
]

# For graphs
CATEGORY_COLORS = [
    '#210c52',  # Underweight
    '#a88de6',  # Normal
    '#d0c2f5',  # Overweight
    '#7a4ec7',  # Obese Class I
    '#5d33ab',  # Obese Class II
    '#3c1e7f'   # Obese Class III 
]

# 16 Selected Features (populated after feature selection)
FEATURE_COLS = [
    'GENHLTH', '_AGEG5YR', 'DIABETE4', '_SEX', 'EMPLOY1', 'SDHFOOD1',
    '_INCOMG1', '_AGE65YR', '_RACE', 'HAVARTH4', '_AGE_G', 'CHILDREN',
    'ALCDAY4', 'EXERANY2', '_TOTINDA', 'MENTHLTH'
]

# Feature descriptions for output tables
FEATURE_NAMES = {
    '_AGEG5YR': 'Age Group (5-year)',
    'GENHLTH': 'General Health',
    'MENTHLTH': 'Mental Health Days',
    'DIABETE4': 'Diabetes Status',
    '_INCOMG1': 'Income Group',
    'EMPLOY1': 'Employment Status',
    'ALCDAY4': 'Alcohol Consumption',
    '_AGE_G': 'Age Detailed',
    '_RACE': 'Race/Ethnicity',
    '_AGE65YR': 'Age 65+',
    'SDHFOOD1': 'Food Security',
    'CHILDREN': 'Number of Children',
    'HAVARTH4': 'Arthritis',
    '_SEX': 'Biological Sex',
    'EXERANY2': 'Any Exercise',
    '_TOTINDA': 'Physical Activity Index'
}

# File paths
BASE_DIR = Path('.')  
DATA_DIR = Path('..') / 'data' 
FIGURE_DIR = BASE_DIR / 'figures'
OUTPUT_DIR = BASE_DIR / 'outputs'
FEATURE_DIR = BASE_DIR / 'feature_selection_results'
ORIGINAL_DATA_PATH = DATA_DIR / 'LLCP2024.csv'
BALANCED_44K_PATH = DATA_DIR / 'brfss_balanced_RANDOM.csv'
BALANCED_100K_PATH = DATA_DIR / 'brfss_balanced_bmi6_100k (2) (1).csv'
FIGURE_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
FEATURE_DIR.mkdir(parents=True, exist_ok=True)

# Apply color palette to matplotlib and seaborn
sns.set_palette(CATEGORY_COLORS)
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=CATEGORY_COLORS)

In [ ]:
def bmi_to_category(bmi):

    if pd.isna(bmi):
        return None
    
    thresholds = [
        (18.5, 'Underweight'),
        (25, 'Normal'),
        (30, 'Overweight'),
        (35, 'Obese Class I'),
        (40, 'Obese Class II'),
        (100, 'Obese Class III')
    ]
    
    for threshold, category in thresholds:
        if bmi < threshold:
            return category
    
    return 'Obese Class III'


def get_bmi_category_numeric(bmi):

    if bmi < 18.5:
        return 1  # Underweight
    elif bmi < 25:
        return 2  # Normal
    elif bmi < 30:
        return 3  # Overweight
    elif bmi < 35:
        return 4  # Obese Class I
    elif bmi < 40:
        return 5  # Obese Class II
    else:
        return 6  # Obese Class III


def print_category_dist(category_counts, total, title="Category Distribution"):

    print(f"\n{title}:")
    print("-" * 60)
    print(f"{'Category':<20} {'Count':>15} {'Percentage':>15}")
    print("-" * 60)
    
    for cat in BMI_CATEGORIES:
        count = category_counts.get(cat, 0) if isinstance(category_counts, dict) else category_counts.get(cat, 0)
        pct = (count / total * 100) if total > 0 else 0
        print(f"{cat:<20} {count:>15,} {pct:>14.2f}%")
    
    print("-" * 60)
    print(f"{'Total':<20} {total:>15,} {100.00:>14.2f}%")


def save_figure(fig, filename, dpi=300, bbox_inches='tight'):

    filepath = FIGURE_DIR / filename
    fig.savefig(filepath, dpi=dpi, bbox_inches=bbox_inches)
    print(f"Saved: {filepath}")


def get_colors(n_colors=5):

    if n_colors <= len(CATEGORY_COLORS):
        return CATEGORY_COLORS[:n_colors]
    else:
        return [CATEGORY_COLORS[i % len(CATEGORY_COLORS)] for i in range(n_colors)]


def create_category_table(category_counts, total):

    bmi_ranges = [
        'BMI < 18.5',
        '18.5 ≤ BMI < 25',
        '25 ≤ BMI < 30',
        '30 ≤ BMI < 35',
        '35 ≤ BMI < 40',
        'BMI ≥ 40'
    ]
    
    data = []
    for cat, bmi_range in zip(BMI_CATEGORIES, bmi_ranges):
        count = category_counts.get(cat, 0) if isinstance(category_counts, dict) else category_counts.get(cat, 0)
        pct = (count / total * 100) if total > 0 else 0
        data.append({
            'Category': cat,
            'BMI Range': bmi_range,
            'Count': count,
            'Percentage': f"{pct:.2f}%"
        })
    
    return pd.DataFrame(data)


---

## Section 2: Data Loading & Preprocessing



In [ ]:
data = pd.read_csv(ORIGINAL_DATA_PATH)

# Convert _BMI5 to BMI since BRFSS stores as integer: 2500 = 25.00
bmi_raw = pd.to_numeric(data['_BMI5'], errors='coerce')

# Check if BMI is stored as integer or already as decimal
if bmi_raw.dropna().max() > 100:
    data['BMI'] = bmi_raw.replace({9999: np.nan}) / 100.0
else:
    data['BMI'] = bmi_raw

# Filter invalid BMI values
data['BMI'] = data['BMI'].mask((data['BMI'] < 12.0) | (data['BMI'] >= 100.0))
data_valid = data[data['BMI'].notna()].copy()

print(f"Valid samples: {len(data_valid):,} (removed {len(data) - len(data_valid):,} invalid/missing)")
print(f"   Mean: {data_valid['BMI'].mean():.2f}")
print(f"   Median: {data_valid['BMI'].median():.2f}")
print(f"   Std: {data_valid['BMI'].std():.2f}")
print(f"   Range: [{data_valid['BMI'].min():.2f}, {data_valid['BMI'].max():.2f}]")

In [ ]:
data_valid['BMI_Category'] = data_valid['BMI'].apply(bmi_to_category)
category_counts = data_valid['BMI_Category'].value_counts()

# Sort by category order 
category_counts_ordered = pd.Series({
    cat: category_counts.get(cat, 0) 
    for cat in BMI_CATEGORIES
})

print_category_dist(category_counts_ordered, len(data_valid), "Original BMI Category Distribution")
imbalance_ratio = category_counts_ordered.max() / category_counts_ordered.min()
print(f"\nImbalance ratio: {imbalance_ratio:.2f}x (largest category is {imbalance_ratio:.2f}x the smallest)")



In [ ]:
table_df = create_category_table(category_counts_ordered, len(data_valid))
table_path = OUTPUT_DIR / 'table1_bmi_category_distribution.csv'
table_df.to_csv(table_path, index=False)

print(f"\nTable saved to: {table_path}")
print("\nBMI Category Distribution")
print("=" * 80)
print(table_df.to_string(index=False))

In [ ]:
counts_ordered = [category_counts_ordered.get(cat, 0) for cat in BMI_CATEGORIES]
pcts_ordered = [(c / len(data_valid) * 100) for c in counts_ordered]

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart
ax1 = axes[0]
bars = ax1.bar(BMI_CATEGORIES, counts_ordered, color=CATEGORY_COLORS, alpha=0.8, edgecolor='black', linewidth=1.5)
ax1.set_xlabel('BMI Category', fontweight='bold', fontsize=12)
ax1.set_ylabel('Number of Samples', fontweight='bold', fontsize=12)
ax1.set_title('Original BMI Category Distribution', fontweight='bold', fontsize=14)
ax1.set_xticklabels(BMI_CATEGORIES, rotation=45, ha='right')
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, count, pct in zip(bars, counts_ordered, pcts_ordered):
    ax1.text(bar.get_x() + bar.get_width()/2., bar.get_height(),
             f'{count:,}\n({pct:.1f}%)', ha='center', va='bottom', fontsize=9, fontweight='bold')

# Pie chart
ax2 = axes[1]
wedges, texts, autotexts = ax2.pie(
    pcts_ordered, 
    labels=BMI_CATEGORIES, 
    colors=CATEGORY_COLORS,
    autopct='%1.1f%%', 
    startangle=90, 
    textprops={'fontsize': 10}
)
ax2.set_title('BMI Category Distribution (%)', fontweight='bold', fontsize=14)

for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

plt.tight_layout()

save_figure(fig, 'original_bmi_category_distribution.png', dpi=300)
plt.show()



---

## Section 3: Dataset Configurations


In [ ]:
# Find minimum available samples per category 
available_samples = {cat: len(data_valid[data_valid['BMI_Category'] == cat]) for cat in BMI_CATEGORIES}
target_per_category = min(available_samples.values())

print(f"\nAvailable samples per category:")
for cat, count in available_samples.items():
    print(f"  {cat:<20}: {count:,}")
print(f"\nTarget per category: {target_per_category:,} (matching smallest: Underweight)")

# Sample equal amounts from each category
balanced_samples_44k = []
for cat in BMI_CATEGORIES:
    cat_data = data_valid[data_valid['BMI_Category'] == cat].sample(n=target_per_category, random_state=42)
    balanced_samples_44k.append(cat_data)

# Combine and shuffle
data_balanced_44k = pd.concat(balanced_samples_44k, ignore_index=True)
data_balanced_44k = data_balanced_44k.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"\nBalanced 44K dataset created: {len(data_balanced_44k):,} samples")
print_category_dist(data_balanced_44k['BMI_Category'].value_counts(), len(data_balanced_44k), "Balanced 44K Distribution")


In [ ]:
# Load the pre-balanced 100K dataset
data_balanced_100k = pd.read_csv(BALANCED_100K_PATH)
if 'BMI_Category' not in data_balanced_100k.columns:
    data_balanced_100k['BMI_Category'] = data_balanced_100k['BMI'].apply(bmi_to_category)

print(f"Loaded 100K dataset: {len(data_balanced_100k):,} samples")
print_category_dist(data_balanced_100k['BMI_Category'].value_counts(), len(data_balanced_100k), "100K Dataset Distribution")


In [ ]:
def prepare_dataset_for_training(df, dataset_name, feature_cols):
    print(f"\nPreparing {dataset_name}...")
    
    available_features = [f for f in feature_cols if f in df.columns]
    missing_features = [f for f in feature_cols if f not in df.columns]
    
    if missing_features:
        print(f"Missing features: {missing_features}")
    
    X = df[available_features].copy()
    y = df['BMI'].copy()
    
    # Handle missing values with median imputation
    imputer = SimpleImputer(strategy='median')
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns, index=X.index)
    
    # Stratified split by BMI category
    y_cat = y.apply(get_bmi_category_numeric)
    X_train, X_test, y_train, y_test = train_test_split(
        X_imputed, y, test_size=0.2, random_state=42, stratify=y_cat
    )
    
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)
    
    print(f"  Train: {len(X_train):,}, Test: {len(X_test):,}")
    
    return {
        'name': dataset_name,
        'X_train': X_train_scaled,
        'X_test': X_test_scaled,
        'y_train': y_train,
        'y_test': y_test,
        'df': df,
        'features': available_features
    }

datasets = {}

# 1. Original (use data_valid from Section 2)
datasets['Original (414K)'] = prepare_dataset_for_training(data_valid, 'Original (414K)', FEATURE_COLS)

# 2. Balanced 44K
datasets['Balanced Fully (44K)'] = prepare_dataset_for_training(data_balanced_44k, 'Balanced Fully (44K)', FEATURE_COLS)

# 3. Even BMI Categories (100K)
datasets['Even BMI Categories (100K)'] = prepare_dataset_for_training(data_balanced_100k, 'Even BMI Categories (100K)', FEATURE_COLS)


In [ ]:
def train_and_evaluate_stacking(dataset_dict):
    name = dataset_dict['name']
    X_train = dataset_dict['X_train']
    X_test = dataset_dict['X_test']
    y_train = dataset_dict['y_train']
    y_test = dataset_dict['y_test']
    
    stacking = StackingRegressor(
        estimators=[
            ('ridge', Ridge(alpha=0.1, random_state=42)),
            ('rf', RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)),
            ('gb', GradientBoostingRegressor(n_estimators=100, learning_rate=0.08, max_depth=10, random_state=42))
        ],
        final_estimator=Ridge(alpha=0.1),
        cv=3,
        n_jobs=-1
    )
    
    stacking.fit(X_train, y_train)
    y_pred = stacking.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"  RMSE: {rmse:.2f}, MAE: {mae:.2f}, R²: {r2:.4f}")
    
    return {
        'name': name,
        'model': stacking,
        'y_pred': y_pred,
        'y_test': y_test,
        'rmse': rmse,
        'mae': mae,
        'r2': r2
    }

results = {}
for ds_name, ds_dict in datasets.items():
    results[ds_name] = train_and_evaluate_stacking(ds_dict)


In [ ]:
DEMOGRAPHIC_FEATURES = {
    '_RACE': {'name': 'Race/Ethnicity', 'labels': {1: 'White', 2: 'Black', 3: 'Native Am', 4: 'Asian',
               5: 'Pacific', 6: 'Other', 7: 'Multiracial', 8: 'Hispanic'}},
    '_SEX': {'name': 'Sex', 'labels': {1: 'Male', 2: 'Female'}},
    '_INCOMG1': {'name': 'Income Level', 'labels': {1: '<$15K', 2: '$15-25K', 3: '$25-35K', 4: '$35-50K',
                   5: '$50-100K', 6: '$100-200K', 7: '>$200K'}},
    '_AGEG5YR': {'name': 'Age Group', 'labels': {1: '18-24', 2: '25-29', 3: '30-34', 4: '35-39', 5: '40-44',
                   6: '45-49', 7: '50-54', 8: '55-59', 9: '60-64', 10: '65-69', 11: '70-74', 12: '75-79', 13: '80+'}}
}

def calculate_disparity(y_true, y_pred, demo_values):
    unique_groups = [g for g in np.unique(demo_values) if pd.notna(g)]
    
    if len(unique_groups) < 2:
        return 0
    
    group_maes = []
    for group in unique_groups:
        mask = demo_values == group
        if mask.sum() >= 10: 
            group_mae = mean_absolute_error(np.array(y_true)[mask], np.array(y_pred)[mask])
            group_maes.append(group_mae)
    
    if len(group_maes) < 2:
        return 0
    
    return max(group_maes) - min(group_maes)

def calculate_avg_disparity(dataset_dict, result):
    df = dataset_dict['df']
    y_test = result['y_test']
    y_pred = result['y_pred']
    
    # Get test indices
    test_idx = y_test.index
    
    disparities = []
    for feat in DEMOGRAPHIC_FEATURES.keys():
        if feat in df.columns:
            demo_values = df.loc[test_idx, feat].values
            disparity = calculate_disparity(y_test.values, y_pred, demo_values)
            if disparity > 0:
                disparities.append(disparity)
    
    return np.mean(disparities) if disparities else 0

# Calculate disparity for each dataset
for ds_name in datasets.keys():
    avg_disp = calculate_avg_disparity(datasets[ds_name], results[ds_name])
    results[ds_name]['avg_disparity'] = avg_disp
    print(f"{ds_name}: Avg Disparity = {avg_disp:.2f}")



In [ ]:
comparison_data = []
for ds_name, result in results.items():
    comparison_data.append({
        'Dataset': ds_name,
        'Samples': len(datasets[ds_name]['df']),
        'MAE': result['mae'],
        'R²': result['r2'],
        'Avg Disparity': result['avg_disparity']
    })

comparison_df = pd.DataFrame(comparison_data)

print("\n" + "=" * 100)
print("TABLE 2: DATASET COMPARISON (Using Stacking Ensemble)")
print("=" * 100)
print(f"\n{'Dataset':<30} {'Samples':>12} {'MAE':>10} {'R²':>10} {'Avg Disparity':>15}")
print("-" * 100)

for _, row in comparison_df.iterrows():
    print(f"{row['Dataset']:<30} {row['Samples']:>12,} {row['MAE']:>10.2f} {row['R²']:>10.4f} {row['Avg Disparity']:>15.2f}")

print("-" * 100)

best_accuracy = comparison_df.loc[comparison_df['MAE'].idxmin(), 'Dataset']
best_fairness = comparison_df.loc[comparison_df['Avg Disparity'].idxmin(), 'Dataset']

print(f"\nBest Accuracy: {best_accuracy} (MAE = {comparison_df['MAE'].min():.2f})")
print(f"Best Fairness: {best_fairness} (Avg Disparity = {comparison_df['Avg Disparity'].min():.2f})")

table_path = OUTPUT_DIR / 'table2_dataset_comparison.csv'
comparison_df.to_csv(table_path, index=False)


In [ ]:
display_names = {
    'Original (414K)': 'Original\n(414K)',
    'Balanced Fully (44K)': 'Even BMI Categories\nvia Undersampling (44K)',
    'Even BMI Categories (100K)': 'Even BMI Categories\nvia Oversampling (100K)'
}

dataset_names = list(results.keys())
display_labels = [display_names.get(name, name) for name in dataset_names]
colors = get_colors(3)

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# R^2 Comparison (bar chart)
ax1 = axes[0]
r2_values = [results[name]['r2'] for name in dataset_names]
bars1 = ax1.bar(display_labels, r2_values, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax1.set_ylabel('R² Score', fontweight='bold', fontsize=11)
ax1.set_title('R² Score by Dataset', fontweight='bold', fontsize=13)
ax1.tick_params(axis='x', rotation=0)
ax1.grid(True, alpha=0.3, axis='y')
for bar, val in zip(bars1, r2_values):
    ax1.text(bar.get_x() + bar.get_width()/2., bar.get_height(), 
             f'{val:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# MAE Comparison (bar chart)
ax2 = axes[1]
mae_values = [results[name]['mae'] for name in dataset_names]
bars2 = ax2.bar(display_labels, mae_values, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax2.set_ylabel('MAE (BMI points)', fontweight='bold', fontsize=11)
ax2.set_title('MAE by Dataset (Lower is Better)', fontweight='bold', fontsize=13)
ax2.tick_params(axis='x', rotation=0)
ax2.grid(True, alpha=0.3, axis='y')
for bar, val in zip(bars2, mae_values):
    ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height(), 
             f'{val:.2f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# Disparity Comparison (bar chart)
ax3 = axes[2]
disp_values = [results[name]['avg_disparity'] for name in dataset_names]
bars3 = ax3.bar(display_labels, disp_values, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax3.set_ylabel('Avg Disparity (BMI points)', fontweight='bold', fontsize=11)
ax3.set_title('Average Disparity by Dataset (Lower is Fairer)', fontweight='bold', fontsize=13)
ax3.tick_params(axis='x', rotation=0) 
ax3.grid(True, alpha=0.3, axis='y')
for bar, val in zip(bars3, disp_values):
    ax3.text(bar.get_x() + bar.get_width()/2., bar.get_height(), 
             f'{val:.2f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
save_figure(fig, 'dataset_comparison.png', dpi=300)
plt.show()

data_selected = data_balanced_100k.copy()


---

## Section 4: Feature Selection

In [ ]:
# Use the selected 100K dataset from Section 3
X = data_selected[FEATURE_COLS].copy()
y = data_selected['BMI'].copy()
y_cat = y.apply(get_bmi_category_numeric)

print(f"Dataset: {len(data_selected):,} samples")
print(f"Features: {len(FEATURE_COLS)}")
print(f"BMI range: {y.min():.1f} - {y.max():.1f}")

# Train/Test Split FIRST (to prevent data leakage)
print("\n[4.1b] Splitting data (80/20, stratified by BMI category)...")
X_train_fs, X_test_fs, y_train_fs, y_test_fs = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y_cat
)

print(f"Training set: {len(X_train_fs):,} samples (80%)")
print(f"Test set: {len(X_test_fs):,} samples (20%)")

# Imputation (fit on train only)
print("\n[4.1c] Imputing missing values...")
imputer_fs = SimpleImputer(strategy='median')
X_train_imp = pd.DataFrame(
    imputer_fs.fit_transform(X_train_fs),
    columns=FEATURE_COLS,
    index=X_train_fs.index
)
X_test_imp = pd.DataFrame(
    imputer_fs.transform(X_test_fs),
    columns=FEATURE_COLS,
    index=X_test_fs.index
)
print(f"Missing values before: {X_train_fs.isnull().sum().sum():,}")
print(f"Missing values after: {X_train_imp.isnull().sum().sum()}")


In [ ]:
# METHOD 1: Correlation (Training data only)
corr_scores = {}
for col in FEATURE_COLS:
    corr = np.abs(np.corrcoef(X_train_imp[col], y_train_fs)[0, 1])
    corr_scores[col] = corr if not np.isnan(corr) else 0
print(f"Correlation calculated on {len(X_train_imp):,} training samples")

# METHOD 2: Mutual Information (Training data only)
mi_scores = mutual_info_regression(X_train_imp, y_train_fs, random_state=42)
mi_dict = dict(zip(FEATURE_COLS, mi_scores))
print(f"MI calculated on {len(X_train_imp):,} training samples")

# METHOD 3: Random Forest Importance (Training data only)
rf_fs = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)
rf_fs.fit(X_train_imp, y_train_fs)
rf_dict = dict(zip(FEATURE_COLS, rf_fs.feature_importances_))
print(f"RF trained on {len(X_train_imp):,} training samples")

# METHOD 4: Permutation Importance (Test data)
perm_result = permutation_importance(rf_fs, X_test_imp, y_test_fs, n_repeats=10, random_state=42, n_jobs=-1)
perm_dict = dict(zip(FEATURE_COLS, perm_result.importances_mean))
print(f"Permutation importance calculated on {len(X_test_imp):,} test samples")


In [ ]:
feature_results = []
for col in FEATURE_COLS:
    feature_results.append({
        'Feature': col,
        'Description': FEATURE_NAMES.get(col, col),
        'Corr': corr_scores[col],
        'MI': mi_dict[col],
        'RF': rf_dict[col],
        'Perm': perm_dict[col]
    })

importance_df = pd.DataFrame(feature_results)

# Normalize each metric to [0, 1] scale
for metric in ['Corr', 'MI', 'RF', 'Perm']:
    min_val = importance_df[metric].min()
    max_val = importance_df[metric].max()
    if max_val > min_val:
        importance_df[metric + '_Norm'] = (importance_df[metric] - min_val) / (max_val - min_val)
    else:
        importance_df[metric + '_Norm'] = 0

# Combined score = average of normalized Corr, MI, RF (excluding Perm for combined)
importance_df['Combined'] = importance_df[['Corr_Norm', 'MI_Norm', 'RF_Norm']].mean(axis=1)

# Sort by Combined score
importance_df = importance_df.sort_values('Combined', ascending=False).reset_index(drop=True)
importance_df['Rank'] = range(1, len(importance_df) + 1)


In [ ]:
table3_df = importance_df[['Rank', 'Feature', 'Description', 'Corr', 'MI', 'RF', 'Combined']].copy()
table3_df = table3_df.round({'Corr': 4, 'MI': 4, 'RF': 4, 'Combined': 3})

print("\n" + "=" * 100)
print("TABLE 3: COMBINED FEATURE IMPORTANCE (All 16 Features)")
print("=" * 100)
print(f"\n{'Rank':<6} {'Feature':<12} {'Description':<25} {'Corr':>8} {'MI':>8} {'RF':>8} {'Combined':>10}")
print("-" * 100)

for _, row in table3_df.iterrows():
    print(f"{int(row['Rank']):<6} {row['Feature']:<12} {row['Description']:<25} "
          f"{row['Corr']:>8.4f} {row['MI']:>8.4f} {row['RF']:>8.4f} {row['Combined']:>10.3f}")

print("-" * 100)

# Save to CSV
table3_path = FEATURE_DIR / 'table3_combined_feature_importance.csv'
table3_df.to_csv(table3_path, index=False)
print(f"\nTable 3 saved to: {table3_path}")


In [ ]:
# Sort by Permutation importance for Table 4
table4_df = importance_df[['Feature', 'Description', 'Perm']].copy()
table4_df = table4_df.sort_values('Perm', ascending=False).reset_index(drop=True)
table4_df['Rank'] = range(1, len(table4_df) + 1)
table4_df = table4_df[['Rank', 'Feature', 'Description', 'Perm']]
table4_df = table4_df.round({'Perm': 4})

print("\n" + "=" * 80)
print("TABLE 4: PERMUTATION FEATURE IMPORTANCE (All 16 Features)")
print("=" * 80)
print(f"\n{'Rank':<6} {'Feature':<12} {'Description':<30} {'Permutation':>12}")
print("-" * 80)

for _, row in table4_df.iterrows():
    print(f"{int(row['Rank']):<6} {row['Feature']:<12} {row['Description']:<30} {row['Perm']:>12.4f}")

print("-" * 80)

table4_path = FEATURE_DIR / 'table4_permutation_feature_importance.csv'
table4_df.to_csv(table4_path, index=False)
print(f"\nTable saved to: {table4_path}")


In [ ]:
# Calculate correlation matrix for selected features + BMI
corr_features = FEATURE_COLS + ['BMI']
corr_data = data_selected[corr_features].copy()

# Impute missing values for correlation calculation
imputer_corr = SimpleImputer(strategy='median')
corr_data_imp = pd.DataFrame(
    imputer_corr.fit_transform(corr_data),
    columns=corr_features
)

# Calculate correlation matrix
corr_matrix = corr_data_imp.corr()

# Create heatmap
fig, ax = plt.subplots(figsize=(14, 12))

from matplotlib.colors import LinearSegmentedColormap
purple_colors = ['#f5f0ff', '#d0c2f5', '#a88de6', '#7a4ec7', '#5d33ab', '#3c1e7f']
purple_cmap = LinearSegmentedColormap.from_list('purple_palette', purple_colors, N=256)

sns.heatmap(
    corr_matrix,
    annot=True,
    fmt='.2f',
    cmap=purple_cmap,
    vmin=-1,
    vmax=1,
    square=True,
    linewidths=0.5,
    ax=ax,
    annot_kws={'size': 8, 'color': 'black'},
    cbar_kws={'shrink': 0.8}
)

ax.set_title('Correlation Matrix: 16 Selected Features + BMI', fontsize=14, fontweight='bold', pad=20)
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right', fontsize=9)
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, fontsize=9)

plt.tight_layout()
save_figure(fig, 'balanced_dataset_correlation_heatmap.png', dpi=300)
plt.show()


In [ ]:
# Sort by Combined importance for plotting
plot_df = importance_df.sort_values('Combined', ascending=True)

fig, ax = plt.subplots(figsize=(12, 8))

# Create horizontal bar chart
colors_bar = get_colors(len(plot_df))
bars = ax.barh(
    range(len(plot_df)), 
    plot_df['Combined'], 
    color=colors_bar[0], 
    alpha=0.8, 
    edgecolor='black',
    linewidth=1
)

ax.set_yticks(range(len(plot_df)))
ax.set_yticklabels([f"{row['Feature']} ({row['Description']})" 
                    for _, row in plot_df.iterrows()], fontsize=9)

for i, (idx, row) in enumerate(plot_df.iterrows()):
    ax.text(row['Combined'] + 0.01, i, f"{row['Combined']:.3f}", 
            va='center', fontsize=9, fontweight='bold')

ax.set_xlabel('Combined Importance Score', fontsize=12, fontweight='bold')
ax.set_title('Feature Importance Rankings (Combined Score)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')
ax.set_xlim(0, plot_df['Combined'].max() * 1.15)

plt.tight_layout()
save_figure(fig, 'feature_importance_bar.png', dpi=300)
plt.show()


---

## Section 5: Model Training & Evaluation


In [ ]:
# Use the selected 100K dataset from Section 3
X = data_selected[FEATURE_COLS].copy()
y_bmi = data_selected['BMI'].copy()
y_category = data_selected['BMI_Category'].copy()
y_obesity = (y_bmi >= 30).astype(int)
y_cat_numeric = y_bmi.apply(get_bmi_category_numeric)

print(f"Dataset: {len(data_selected):,} samples")
print(f"Features: {len(FEATURE_COLS)}")
print(f"Obesity rate: {y_obesity.mean():.1%}")

# Train/Test Split (stratified by BMI category)
X_train, X_test, y_train, y_test, y_cat_train, y_cat_test, y_obs_train, y_obs_test = train_test_split(
    X, y_bmi, y_category, y_obesity,
    test_size=0.2, random_state=42, stratify=y_cat_numeric
)

imputer = SimpleImputer(strategy='median')
X_train_imp = pd.DataFrame(imputer.fit_transform(X_train), columns=FEATURE_COLS, index=X_train.index)
X_test_imp = pd.DataFrame(imputer.transform(X_test), columns=FEATURE_COLS, index=X_test.index)

scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_imp), columns=FEATURE_COLS, index=X_train.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_imp), columns=FEATURE_COLS, index=X_test.index)


In [ ]:
models = {}

# Ridge Regression
ridge = Ridge(alpha=0.1, random_state=42)
ridge.fit(X_train_scaled, y_train)
models['Ridge Regression'] = ridge
print("Ridge trained")

# Lasso Regression
lasso = Lasso(alpha=0.01, random_state=42, max_iter=5000)
lasso.fit(X_train_scaled, y_train)
models['Lasso Regression'] = lasso
print("Lasso trained")

# Decision Tree
dt = DecisionTreeRegressor(max_depth=15, random_state=42)
dt.fit(X_train_scaled, y_train)
models['Decision Tree'] = dt
print("Decision Tree trained")

# Random Forest
rf = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)
rf.fit(X_train_scaled, y_train)
models['Random Forest'] = rf
print("Random Forest trained")

# Gradient Boosting
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.08, max_depth=10, random_state=42)
gb.fit(X_train_scaled, y_train)
models['Gradient Boosting'] = gb
print("Gradient Boosting trained")

# Voting Ensemble
voting = VotingRegressor(estimators=[
    ('ridge', Ridge(alpha=0.1, random_state=42)),
    ('rf', RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)),
    ('gb', GradientBoostingRegressor(n_estimators=100, learning_rate=0.08, max_depth=10, random_state=42))
])
voting.fit(X_train_scaled, y_train)
models['Voting Ensemble'] = voting
print("Voting Ensemble trained")

# Stacking Ensemble
stacking = StackingRegressor(
    estimators=[
        ('ridge', Ridge(alpha=0.1, random_state=42)),
        ('rf', RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1)),
        ('gb', GradientBoostingRegressor(n_estimators=100, learning_rate=0.08, max_depth=10, random_state=42))
    ],
    final_estimator=Ridge(alpha=0.1),
    cv=5,
    n_jobs=-1
)
stacking.fit(X_train_scaled, y_train)
models['Stacking Ensemble'] = stacking
print("Stacking Ensemble trained")


In [ ]:
regression_results = []

for name, model in models.items():

    y_pred = model.predict(X_test_scaled)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    regression_results.append({
        'Model': name,
        'RMSE': rmse,
        'MAE': mae,
        'R2': r2
    })
    
    print(f"  {name}: RMSE={rmse:.2f}, MAE={mae:.2f}, R²={r2:.4f}")

# Add Baseline
y_mean = np.full_like(y_test, y_train.mean())
baseline_rmse = np.sqrt(mean_squared_error(y_test, y_mean))
baseline_mae = mean_absolute_error(y_test, y_mean)
baseline_r2 = 0.0 

regression_results.append({
    'Model': 'Mean Baseline',
    'RMSE': baseline_rmse,
    'MAE': baseline_mae,
    'R2': baseline_r2
})
print(f"  Mean Baseline: RMSE={baseline_rmse:.2f}, MAE={baseline_mae:.2f}, R²={baseline_r2:.4f}")

regression_df = pd.DataFrame(regression_results)
regression_df = regression_df.sort_values('R2', ascending=False).reset_index(drop=True)


In [ ]:
print("\n" + "=" * 70)
print("TABLE 5: REGRESSION-BASED BMI PREDICTION METRICS")
print("=" * 70)
print(f"\n{'Model':<25} {'RMSE':>10} {'MAE':>10} {'R²':>10}")
print("-" * 70)

for _, row in regression_df.iterrows():
    print(f"{row['Model']:<25} {row['RMSE']:>10.2f} {row['MAE']:>10.2f} {row['R2']:>10.4f}")

print("-" * 70)

best_model = regression_df.iloc[0]
print(f"\nBest Model: {best_model['Model']} (R^2 = {best_model['R2']:.4f})")

table5_path = OUTPUT_DIR / 'table5_regression_metrics.csv'
regression_df.to_csv(table5_path, index=False)
print(f"\nTable saved to: {table5_path}")


In [ ]:
plot_df = regression_df[regression_df['Model'] != 'Mean Baseline'].copy()
plot_df = plot_df.sort_values('R2', ascending=True)
model_names = plot_df['Model'].tolist()
colors = get_colors(len(model_names))
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# R^2 Score
ax1 = axes[0]
bars1 = ax1.barh(model_names, plot_df['R2'], color=colors, alpha=0.8, edgecolor='black', linewidth=1)
ax1.set_xlabel('R² Score', fontweight='bold', fontsize=11)
ax1.set_title('R² Score by Model (Higher is Better)', fontweight='bold', fontsize=12)
ax1.grid(True, alpha=0.3, axis='x')
for i, (bar, val) in enumerate(zip(bars1, plot_df['R2'])):
    ax1.text(val + 0.01, bar.get_y() + bar.get_height()/2, f'{val:.4f}', 
             va='center', fontsize=9, fontweight='bold')

# MAE
ax2 = axes[1]
bars2 = ax2.barh(model_names, plot_df['MAE'], color=colors, alpha=0.8, edgecolor='black', linewidth=1)
ax2.set_xlabel('MAE (BMI points)', fontweight='bold', fontsize=11)
ax2.set_title('MAE by Model (Lower is Better)', fontweight='bold', fontsize=12)
ax2.grid(True, alpha=0.3, axis='x')
for i, (bar, val) in enumerate(zip(bars2, plot_df['MAE'])):
    ax2.text(val + 0.05, bar.get_y() + bar.get_height()/2, f'{val:.2f}', 
             va='center', fontsize=9, fontweight='bold')

# RMSE
ax3 = axes[2]
bars3 = ax3.barh(model_names, plot_df['RMSE'], color=colors, alpha=0.8, edgecolor='black', linewidth=1)
ax3.set_xlabel('RMSE (BMI points)', fontweight='bold', fontsize=11)
ax3.set_title('RMSE by Model (Lower is Better)', fontweight='bold', fontsize=12)
ax3.grid(True, alpha=0.3, axis='x')
for i, (bar, val) in enumerate(zip(bars3, plot_df['RMSE'])):
    ax3.text(val + 0.05, bar.get_y() + bar.get_height()/2, f'{val:.2f}', 
             va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
save_figure(fig, 'model_regression_comparison.png', dpi=300)
plt.show()


In [ ]:
model_predictions = {}
for name, model in models.items():
    model_predictions[name] = model.predict(X_test_scaled)

# Store the best model (Stacking Ensemble) predictions separately
best_model_name = regression_df.iloc[0]['Model']
y_pred_best = model_predictions[best_model_name]

# Create predicted categories for classification metrics
def bmi_to_category_name(bmi):
    if bmi < 18.5: return 'Underweight'
    elif bmi < 25: return 'Normal'
    elif bmi < 30: return 'Overweight'
    elif bmi < 35: return 'Obese Class I'
    elif bmi < 40: return 'Obese Class II'
    else: return 'Obese Class III'

y_pred_categories = pd.Series([bmi_to_category_name(bmi) for bmi in y_pred_best], index=y_test.index)


---

## Section 6: Classification & Risk Estimation

In [ ]:
def bmi_to_risk_interpolated(bmi):

    if bmi < 18.5:
        risk = 0.10 + 0.05 * max(0, (18.5 - bmi) / 3.5)
    elif bmi < 25:
        distance_from_ideal = abs(bmi - 22)
        risk = 0.10 + 0.10 * (distance_from_ideal / 3)
    elif bmi < 30:
        position = (bmi - 25) / 5
        risk = 0.30 + 0.20 * position
    elif bmi < 35:
        position = (bmi - 30) / 5
        risk = 0.60 + 0.20 * position
    elif bmi < 40:
        position = (bmi - 35) / 5
        risk = 0.80 + 0.12 * position
    else:
        excess = min((bmi - 40) / 10, 1)
        risk = 0.92 + 0.07 * excess
    return np.clip(risk, 0, 1)

CATEGORY_ORDER = ['Underweight', 'Normal', 'Overweight', 'Obese Class I', 'Obese Class II', 'Obese Class III']


In [ ]:
classification_results = {}

for name, model in models.items():

    y_bmi_pred = model_predictions[name]
    y_category_pred = pd.Series([bmi_to_category_name(bmi) for bmi in y_bmi_pred])
    y_risk_proba = np.array([bmi_to_risk_interpolated(bmi) for bmi in y_bmi_pred])
    y_obesity_pred = (y_bmi_pred >= 30).astype(int)
    
    accuracy = accuracy_score(y_obs_test, y_obesity_pred)
    precision = precision_score(y_obs_test, y_obesity_pred, zero_division=0)
    recall = recall_score(y_obs_test, y_obesity_pred, zero_division=0)
    f1 = f1_score(y_obs_test, y_obesity_pred, zero_division=0)
    auc_roc = roc_auc_score(y_obs_test, y_risk_proba)
    brier = brier_score_loss(y_obs_test, y_risk_proba)
    
    category_accuracy = accuracy_score(y_cat_test, y_category_pred)
    actual_cat_idx = [CATEGORY_ORDER.index(cat) for cat in y_cat_test]
    pred_cat_idx = [CATEGORY_ORDER.index(cat) for cat in y_category_pred]
    within_1_category = (np.abs(np.array(actual_cat_idx) - np.array(pred_cat_idx)) <= 1).mean()
    
    bmi_tolerance_2 = (np.abs(y_bmi_pred - y_test.values) <= 2.0).mean()
    bmi_tolerance_3 = (np.abs(y_bmi_pred - y_test.values) <= 3.0).mean()
    
    classification_results[name] = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc_roc': auc_roc,
        'brier': brier,
        'category_accuracy': category_accuracy,
        'within_1_category': within_1_category,
        'bmi_tolerance_2': bmi_tolerance_2,
        'bmi_tolerance_3': bmi_tolerance_3,
        'y_risk_proba': y_risk_proba,
        'y_obesity_pred': y_obesity_pred
    }
    

In [ ]:
table6_data = []
for name, res in classification_results.items():
    table6_data.append({
        'Model': name,
        'Accuracy': res['accuracy'],
        'Precision': res['precision'],
        'Recall': res['recall'],
        'F1': res['f1'],
        'AUC-ROC': res['auc_roc']
    })

table6_df = pd.DataFrame(table6_data)
table6_df = table6_df.sort_values('AUC-ROC', ascending=False).reset_index(drop=True)

print("\n" + "=" * 90)
print("TABLE 6: BINARY OBESITY CLASSIFICATION (BMI ≥ 30)")
print("=" * 90)
print(f"\n{'Model':<25} {'Accuracy':>10} {'Precision':>10} {'Recall':>10} {'F1':>10} {'AUC-ROC':>10}")
print("-" * 90)

for _, row in table6_df.iterrows():
    print(f"{row['Model']:<25} {row['Accuracy']:>10.4f} {row['Precision']:>10.4f} "
          f"{row['Recall']:>10.4f} {row['F1']:>10.4f} {row['AUC-ROC']:>10.4f}")

print("-" * 90)

# Best model
best_auc = table6_df.iloc[0]
print(f"\nBest AUC-ROC: {best_auc['Model']} ({best_auc['AUC-ROC']:.4f})")

table6_path = OUTPUT_DIR / 'table6_binary_classification.csv'
table6_df.to_csv(table6_path, index=False)
print(f"\nTable saved to: {table6_path}")


In [ ]:
table7_data = []
for name, res in classification_results.items():
    table7_data.append({
        'Model': name,
        'Category Acc': res['category_accuracy'],
        'Within-1-Cat': res['within_1_category'],
        '±2 BMI': res['bmi_tolerance_2'],
        '±3 BMI': res['bmi_tolerance_3'],
        'Brier Score': res['brier']
    })

table7_df = pd.DataFrame(table7_data)
table7_df = table7_df.sort_values('Brier Score', ascending=True).reset_index(drop=True)

print("\n" + "=" * 90)
print("TABLE 7: RISK ESTIMATION & CATEGORY PREDICTION")
print("=" * 90)
print(f"\n{'Model':<25} {'Cat-Acc':>10} {'±1-Cat':>10} {'±2 BMI':>10} {'±3 BMI':>10} {'Brier':>10}")
print("-" * 90)

for _, row in table7_df.iterrows():
    print(f"{row['Model']:<25} {row['Category Acc']:>10.4f} {row['Within-1-Cat']:>10.4f} "
          f"{row['±2 BMI']:>10.4f} {row['±3 BMI']:>10.4f} {row['Brier Score']:>10.4f}")

print("-" * 90)
# Best model
best_brier = table7_df.iloc[0]
print(f"\nBest Brier Score: {best_brier['Model']} ({best_brier['Brier Score']:.4f})")

table7_path = OUTPUT_DIR / 'table7_risk_estimation.csv'
table7_df.to_csv(table7_path, index=False)
print(f"\nTable saved to: {table7_path}")


In [ ]:
metrics = ['Accuracy', 'Precision', 'Recall', 'F1', 'AUC-ROC']
model_names = list(classification_results.keys())

# Create radar chart data
radar_data = pd.DataFrame({
    name: [classification_results[name]['accuracy'], 
           classification_results[name]['precision'],
           classification_results[name]['recall'], 
           classification_results[name]['f1'], 
           classification_results[name]['auc_roc']]
    for name in model_names
}, index=metrics)

num_vars = len(metrics)
angles = [n / float(num_vars) * 2 * np.pi for n in range(num_vars)]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
colors = get_colors(len(model_names))

for i, name in enumerate(model_names):
    values = radar_data[name].tolist()
    values += values[:1]
    
    ax.plot(angles, values, 'o-', linewidth=2, label=name, color=colors[i % len(colors)])
    ax.fill(angles, values, alpha=0.1, color=colors[i % len(colors)])
ax.set_theta_offset(np.pi / 2)
ax.set_theta_direction(-1)
ax.set_xticks(angles[:-1])
ax.set_xticklabels(metrics, fontsize=11, fontweight='bold')
ax.set_ylim(0, 1)
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], fontsize=9)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0), fontsize=9)

ax.set_title('Model Performance Comparison\n(Binary Obesity Classification)', 
             fontsize=14, fontweight='bold', y=1.08)

plt.tight_layout()
save_figure(fig, 'model_performance_radar.png', dpi=300)
plt.show()

print("\nRadar chart saved")


---

## Section 7: Fairness Analysis

In [ ]:
from collections import defaultdict

demographic_groups = {
    '_SEX': {
        1: 'Male',
        2: 'Female'
    },
    '_AGEG5YR': {
        'Young Adult (18-39)': lambda x: x <= 5,
        'Middle Age (40-59)': lambda x: (x > 5) & (x <= 9),
        'Senior (60+)': lambda x: x > 9
    },
    '_RACE': {
        'White': lambda x: x == 1,
        'Black': lambda x: x == 2,
        'Asian': lambda x: x == 4,
        'Hispanic': lambda x: x == 8,
        'Other': lambda x: (x != 1) & (x != 2) & (x != 4) & (x != 8)
    },
    '_INCOMG1': {
        'Low Income (<$25k)': lambda x: x <= 2,
        'Middle Income ($25-75k)': lambda x: (x > 2) & (x <= 5),
        'High Income ($75k+)': lambda x: x > 5
    }
}

best_bmi_pred = model_predictions[best_model_name]
best_obesity_pred = (best_bmi_pred >= 30).astype(int)
best_risk_proba = np.array([bmi_to_risk_interpolated(bmi) for bmi in best_bmi_pred])


In [ ]:
fairness_results = defaultdict(dict)
X_test_orig = X_test_imp.copy()

for sex_code, sex_name in demographic_groups['_SEX'].items():
    mask = X_test_orig['_SEX'] == sex_code
    if mask.sum() == 0:
        continue
    
    n = mask.sum()
    mae = mean_absolute_error(y_test[mask], best_bmi_pred[mask])
    acc = accuracy_score(y_obs_test[mask], best_obesity_pred[mask])
    prec = precision_score(y_obs_test[mask], best_obesity_pred[mask], zero_division=0)
    rec = recall_score(y_obs_test[mask], best_obesity_pred[mask], zero_division=0)
    auc = roc_auc_score(y_obs_test[mask], best_risk_proba[mask])
    
    fairness_results['SEX'][sex_name] = {
        'n': n, 'mae': mae, 'accuracy': acc, 'precision': prec, 'recall': rec, 'auc_roc': auc
    }

for age_name, age_func in demographic_groups['_AGEG5YR'].items():
    mask = age_func(X_test_orig['_AGEG5YR'])
    if mask.sum() == 0:
        continue
    
    n = mask.sum()
    mae = mean_absolute_error(y_test[mask], best_bmi_pred[mask])
    acc = accuracy_score(y_obs_test[mask], best_obesity_pred[mask])
    prec = precision_score(y_obs_test[mask], best_obesity_pred[mask], zero_division=0)
    rec = recall_score(y_obs_test[mask], best_obesity_pred[mask], zero_division=0)
    auc = roc_auc_score(y_obs_test[mask], best_risk_proba[mask])
    
    fairness_results['AGE'][age_name] = {
        'n': n, 'mae': mae, 'accuracy': acc, 'precision': prec, 'recall': rec, 'auc_roc': auc
    }

for race_name, race_func in demographic_groups['_RACE'].items():
    mask = race_func(X_test_orig['_RACE'])
    if mask.sum() < 100: 
        continue
    
    n = mask.sum()
    mae = mean_absolute_error(y_test[mask], best_bmi_pred[mask])
    acc = accuracy_score(y_obs_test[mask], best_obesity_pred[mask])
    prec = precision_score(y_obs_test[mask], best_obesity_pred[mask], zero_division=0)
    rec = recall_score(y_obs_test[mask], best_obesity_pred[mask], zero_division=0)
    auc = roc_auc_score(y_obs_test[mask], best_risk_proba[mask])
    
    fairness_results['RACE'][race_name] = {
        'n': n, 'mae': mae, 'accuracy': acc, 'precision': prec, 'recall': rec, 'auc_roc': auc
    }

for income_name, income_func in demographic_groups['_INCOMG1'].items():
    mask = income_func(X_test_orig['_INCOMG1'])
    if mask.sum() == 0:
        continue
    
    n = mask.sum()
    mae = mean_absolute_error(y_test[mask], best_bmi_pred[mask])
    acc = accuracy_score(y_obs_test[mask], best_obesity_pred[mask])
    prec = precision_score(y_obs_test[mask], best_obesity_pred[mask], zero_division=0)
    rec = recall_score(y_obs_test[mask], best_obesity_pred[mask], zero_division=0)
    auc = roc_auc_score(y_obs_test[mask], best_risk_proba[mask])
    
    fairness_results['INCOME'][income_name] = {
        'n': n, 'mae': mae, 'accuracy': acc, 'precision': prec, 'recall': rec, 'auc_roc': auc
    }

In [ ]:
disparity_summary = []

for demo_name, demo_results in fairness_results.items():
    if len(demo_results) < 2:
        continue
    
    maes = [v['mae'] for v in demo_results.values()]
    accs = [v['accuracy'] for v in demo_results.values()]
    aucs = [v['auc_roc'] for v in demo_results.values()]
    
    mae_disparity = max(maes) - min(maes)
    acc_disparity = max(accs) - min(accs)
    auc_disparity = max(aucs) - min(aucs)
    
    # Find best and worst groups
    best_group = min(demo_results.keys(), key=lambda k: demo_results[k]['mae'])
    worst_group = max(demo_results.keys(), key=lambda k: demo_results[k]['mae'])
    
    disparity_summary.append({
        'Demographic': demo_name,
        'MAE Disparity': mae_disparity,
        'Accuracy Disparity': acc_disparity,
        'AUC Disparity': auc_disparity,
        'Best Group (MAE)': best_group,
        'Worst Group (MAE)': worst_group,
        'Best MAE': min(maes),
        'Worst MAE': max(maes)
    })
    
    print(f"\n  {demo_name}:")
    print(f"    MAE Range: {min(maes):.2f} - {max(maes):.2f} (Δ = {mae_disparity:.2f})")
    print(f"    Best: {best_group}, Worst: {worst_group}")

disparity_df = pd.DataFrame(disparity_summary)
disparity_df = disparity_df.sort_values('MAE Disparity', ascending=False).reset_index(drop=True)


In [ ]:
print("\n" + "=" * 100)
print("TABLE 8: DEMOGRAPHIC DISPARITY ANALYSIS")
print("=" * 100)
print(f"\n{'Demographic':<12} {'MAE Δ':>10} {'Acc Δ':>10} {'AUC Δ':>10} {'Best Group':<25} {'Worst Group':<25}")
print("-" * 100)

for _, row in disparity_df.iterrows():
    print(f"{row['Demographic']:<12} {row['MAE Disparity']:>10.2f} {row['Accuracy Disparity']:>10.4f} "
          f"{row['AUC Disparity']:>10.4f} {row['Best Group (MAE)']:<25} {row['Worst Group (MAE)']:<25}")

print("-" * 100)

primary_bias = disparity_df.iloc[0]
print(f"\nPRIMARY SOURCE OF BIAS: {primary_bias['Demographic']} (MAE Disparity: {primary_bias['MAE Disparity']:.2f} BMI points)")

table8_path = OUTPUT_DIR / 'table8_demographic_disparity.csv'
disparity_df.to_csv(table8_path, index=False)
print(f"\nTable saved to: {table8_path}")


In [ ]:
model_fairness = []

for name in models.keys():
    y_pred = model_predictions[name]
    y_obesity_pred_model = (y_pred >= 30).astype(int)

    overall_mae = mean_absolute_error(y_test, y_pred)
    overall_r2 = regression_df[regression_df['Model'] == name]['R2'].values[0]
    
    # Calculate income disparity (primary bias source)
    income_maes = []
    for income_name, income_func in demographic_groups['_INCOMG1'].items():
        mask = income_func(X_test_orig['_INCOMG1'])
        if mask.sum() > 0:
            income_mae = mean_absolute_error(y_test[mask], y_pred[mask])
            income_maes.append(income_mae)
    
    income_disparity = max(income_maes) - min(income_maes) if len(income_maes) >= 2 else 0
    
    all_disparities = []
    for demo_name in ['_SEX', '_AGEG5YR', '_RACE', '_INCOMG1']:
        demo_maes = []
        if demo_name == '_SEX':
            for code, _ in demographic_groups['_SEX'].items():
                mask = X_test_orig['_SEX'] == code
                if mask.sum() > 0:
                    demo_maes.append(mean_absolute_error(y_test[mask], y_pred[mask]))
        else:
            for _, func in demographic_groups[demo_name].items():
                mask = func(X_test_orig[demo_name])
                if mask.sum() > 100:
                    demo_maes.append(mean_absolute_error(y_test[mask], y_pred[mask]))
        
        if len(demo_maes) >= 2:
            all_disparities.append(max(demo_maes) - min(demo_maes))
    
    avg_disparity = np.mean(all_disparities) if all_disparities else 0
    
    model_fairness.append({
        'Model': name,
        'MAE': overall_mae,
        'R2': overall_r2,
        'Income Disparity': income_disparity,
        'Avg Disparity': avg_disparity
    })

fairness_trade_off_df = pd.DataFrame(model_fairness)
fairness_trade_off_df = fairness_trade_off_df.sort_values('Avg Disparity', ascending=True).reset_index(drop=True)

# Print trade-off table
print("\n" + "=" * 80)
print("ACCURACY-FAIRNESS TRADE-OFF BY MODEL")
print("=" * 80)
print(f"\n{'Model':<25} {'MAE':>8} {'R²':>8} {'Inc Δ':>10} {'Avg Δ':>10}")
print("-" * 80)

for _, row in fairness_trade_off_df.iterrows():
    print(f"{row['Model']:<25} {row['MAE']:>8.2f} {row['R2']:>8.4f} "
          f"{row['Income Disparity']:>10.2f} {row['Avg Disparity']:>10.2f}")

print("-" * 80)

# Find fairest and most accurate models
fairest_model = fairness_trade_off_df.iloc[0]['Model']
most_accurate = fairness_trade_off_df.sort_values('R2', ascending=False).iloc[0]['Model']
print(f"\nMost Accurate: {most_accurate}")
print(f"Fairest Model: {fairest_model}")

tradeoff_path = OUTPUT_DIR / 'accuracy_fairness_tradeoff.csv'
fairness_trade_off_df.to_csv(tradeoff_path, index=False)
print(f"\nTrade-off analysis saved to: {tradeoff_path}")


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# MAE Disparity by Demographic
ax1 = axes[0]
demo_names = disparity_df['Demographic'].tolist()
mae_disparities = disparity_df['MAE Disparity'].tolist()
colors = get_colors(len(demo_names))
bars1 = ax1.bar(demo_names, mae_disparities, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
ax1.set_ylabel('MAE Disparity (BMI points)', fontweight='bold', fontsize=11)
ax1.set_title('MAE Disparity by Demographic', fontweight='bold', fontsize=12)
ax1.tick_params(axis='x', rotation=0)
ax1.grid(True, alpha=0.3, axis='y')
for bar, val in zip(bars1, mae_disparities):
    ax1.text(bar.get_x() + bar.get_width()/2., bar.get_height(), f'{val:.2f}', 
             ha='center', va='bottom', fontsize=10, fontweight='bold')

# MAE by Income Group (showing primary bias)
ax2 = axes[1]
if 'INCOME' in fairness_results:
    income_groups = list(fairness_results['INCOME'].keys())
    income_maes = [fairness_results['INCOME'][g]['mae'] for g in income_groups]
    short_names = ['Low\n(<$25k)', 'Middle\n($25-75k)', 'High\n($75k+)']
    bars2 = ax2.bar(short_names, income_maes, color=colors[:3], alpha=0.8, edgecolor='black', linewidth=1.5)
    ax2.set_ylabel('MAE (BMI points)', fontweight='bold', fontsize=11)
    ax2.set_title('MAE by Income Group\n(Primary Bias Source)', fontweight='bold', fontsize=12)
    ax2.grid(True, alpha=0.3, axis='y')
    for bar, val in zip(bars2, income_maes):
        ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height(), f'{val:.2f}', 
                 ha='center', va='bottom', fontsize=10, fontweight='bold')

# Accuracy-Fairness Trade-off
ax3 = axes[2]
for i, (_, row) in enumerate(fairness_trade_off_df.iterrows()):
    ax3.scatter(row['MAE'], row['Avg Disparity'], s=200, c=[colors[i % len(colors)]], 
                alpha=0.8, edgecolors='black', linewidth=1.5, label=row['Model'])
    ax3.annotate(row['Model'].split()[0], (row['MAE'], row['Avg Disparity']), 
                 textcoords="offset points", xytext=(5, 5), fontsize=8)

ax3.set_xlabel('MAE (Lower = More Accurate)', fontweight='bold', fontsize=11)
ax3.set_ylabel('Avg Disparity (Lower = Fairer)', fontweight='bold', fontsize=11)
ax3.set_title('Accuracy-Fairness Trade-off', fontweight='bold', fontsize=12)
ax3.grid(True, alpha=0.3)

plt.tight_layout()
save_figure(fig, 'fairness_disparity_analysis.png', dpi=300)
plt.show()
